In [ ]:
## Author : Seong-Han

import copy
import json
import random
import io
import distutils.dir_util
import warnings
warnings.filterwarnings("ignore")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
        
from collections import Counter
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')
import os
os.chdir('gdrive/My Drive/Google Colaboratory/kakao_arena/kakao_arena_melon/src/ipynb')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
import sys
sys.path.append('../')
from basic_utils import *

In [16]:
data_path = './../../../1_melon_playlist/data/'
train = pd.read_json(data_path+'train.json')
val = pd.read_json(data_path+'val.json')
# test = pd.read_json(data_path+'test.json')
song_meta = load_json(data_path+'song_meta.json')
# genre_gn_all = pd.read_json(data_path+'genre_gn_all.json', typ = 'series')

### make_file.ipynb 내에서 만들어진 tag_song_dict / song_tag_dict 불러오기
tag_song_dict = load_json(data_path+'tag_song_dict_no0.json')
song_tag_dict = load_json(data_path+'song_tag_dict.json')

In [ ]:
train['date'] = train['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )    # 연도+월
train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))                                    # 연도

date_list = list(train['date'].unique())
date_list= sorted(date_list)

date_dict = {}       # 해당 연도+월 에 index 부여(?)
for i,k in enumerate(date_list):
    date_dict[k] = i

In [18]:
#### popular monthly tag
popular_date_tag = []
for i in range(len(date_list)):
    ## if 부분 재검토 해보기
    if i <= 13:  ## 13이전에는 한 달에 100개가 안나옴
        year = int(str(date_list[i])[0]) ## year
        tmp = train[train['year'] == year]
        _, year_tag = most_popular(tmp, "tags", 100)
        popular_date_tag.append({
            "date" : date_list[i],
            "tags" : year_tag,})
    else:
        tmp = train[train['date'] == date_list[i]]
        _, date_tag = most_popular(tmp, "tags", 100)
        popular_date_tag.append({
            "date" : date_list[i],
            "tags" : date_tag,})

In [25]:
#### popular monthly song
popular_date_song = []
for i in range(len(date_list)):
    ## if 부분 재검토 해보기
    if i <= 9:  ## 11이전에는 한 달에 100개가 안나옴
        year = int(str(date_list[i])[0]) ## year
        tmp = train[train['year'] == year]
        _, date_song = most_popular(tmp, "songs", 200)
        popular_date_song.append({
            "date" : date_list[i],
            "songs" : date_song,})               
    else:
        tmp = train[train['date'] == date_list[i]]
        _, date_song = most_popular(tmp, "songs", 200)
        popular_date_song.append({
            "date" : date_list[i],
            "songs" : date_song,})

### predict validation

In [26]:
val['date'] = val['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )
val['year'] = val['updt_date'].apply(lambda x: int(x[2:4]))

no_tag, no_song, yes_index, no_both = check_target_type(val)

노래 O 태그 X : 9661개
노래 X 태그 O : 2630개
노래 O 태그 O : 8975개
노래 X 태그 X : 1749개


#### val2 : 노래가 없는 케이스

In [27]:
val2 = val[val.index.isin(no_song)]
val2.reset_index(inplace = True, drop = True)
val2.head()

,tags,id,plylst_title,songs,like_cnt,updt_date,date,year
0,[스트레스],80810,리듬타면서 빡시게 운동하자!!!(스트레스 날리자):},[],127,2017-02-09 17:33:45.000,1702,17
1,[생각나],2380,다시 생각나는 그 사람,[],6,2016-03-19 23:53:20.000,1603,16
2,"[피아노, 이루마, 메로디]",149069,불면증엔 아름다운 뉴에이지 곡들,[],11,2019-12-15 19:15:21.000,1912,19
3,"[발렌타인데이, 달달한, 사랑노래, 고백]",80682,발렌타인 데이 -달콤한 초콜릿 매장 음악,[],27,2019-05-13 15:46:52.000,1905,19
4,[새벽],8253,하루 끝 잔잔한 노래로 기분 좋게 잠에 들기,[],8,2019-06-08 23:48:41.000,1906,19


In [ ]:
val2_predict = []
for i in val2.index:
    date = date_dict[val2.loc[i]['date']]
    #####  song 채우기
    song_counter = Counter()
    for tag in list(val2.loc[i:i, 'tags'].values[0]):
        if str(tag) in tag_song_dict:
            for song in tag_song_dict[str(tag)]:
                song_counter.update({song : 1})
    song_counter = sorted(song_counter.items(), key= lambda x:x[1], reverse = True)        
    cur_song = []
    for k in song_counter[:100]:
        cur_song.append(k[0])
    if len(cur_song) == 0:
        cur_song = popular_date_song[date]['songs'][:100]
    elif len(cur_song) < 100:
        update_song = remove_seen(cur_song, popular_date_song[date]['songs'])
        cur_song.extend(update_song)
        cur_song = cur_song[:100]
    ##### tag 채우기
    tag_counter = Counter()
    for song in val2.loc[i]['songs']:
        if str(song) in song_tag_dict:
            for tag in song_tag_dict[str(song)]:
                tag_counter.update({tag : 1})
    tag_counter = sorted(tag_counter.items(), key= lambda x:x[1], reverse = True)        
    tag_counter = tag_counter[:100] ## 넉넉히 100개 잡는다. 
    cur_tag = []
    for k in tag_counter:
        cur_tag.append(k[0])     
    if len(cur_tag) == 0:
        cur_tag = popular_date_tag[date]['tags'][:100]
    elif len(cur_tag) < 100:
        update_tag = remove_seen(cur_tag, popular_date_tag[date]['tags'])
        cur_tag.extend(update_tag)
        cur_tag = cur_tag[:100]

    val2_predict.append({
        "id" : list(val2.loc[i:i, 'id'])[0],
        "songs": cur_song,
        "tags": remove_seen(list(val2.loc[i:i, 'tags'])[0], cur_tag)[:10],
    })

In [ ]:
## save file
write_json(val2_predict, 'val2_predict.json')

#### val4 : 태그, 노래 모두 없는 케이스
- 카카오 아레나의 방식으로 추천
- 각 방식마다 나눠서 학습해야하지만 부분적인 성능 확인을 위해서 일단 하나씩 확인

In [ ]:
# Validate = val[~val.index.isin(no_tag)]
# Validate = Validate[~Validate.index.isin(no_song)]
# Validate = Validate[~Validate.index.isin(yes_index)]
# Validate.reset_index(inplace = True, drop = True)

In [ ]:
val4 = val[val.index.isin(no_both)]
val4_predict = []

for i in val4.index:
    date = date_dict[val4.loc[i]['date']]
    val_predict.append({
        "id" : list(Validate.loc[i:i, 'id'])[0],
        "songs": popular_date_song[date]['songs'][:100],
        "tags": popular_date_tag[date]['tags'][:10],
    })

In [ ]:
write_json(val_predict, 'val4_predict.json')